**Fecha de elaboración: 02 febrero de 2022**
# Prediciendo el precio de venta de los inmuebles para vivienda en Bogotá, Colombia

Datos provistos por <a href='https://www.properati.com.co'>Properati</a>

**Contexto:**
La base de datos contiene los anuncios de diferentes predios registrados en el portal inmobiliario de Properati para Colombia. Incluye varios tipos de propiedades (casa, apartamentos, oficinas, locales, etc.) que se ofrecen para la venta o arriendo en las diferentes ciudades y municipios del país. Además, para cada propiedad ofrece información sobre las características del predio como el área, cantidad de habitaciones, precio, entre otros.


**Objetivo de la actividad:**
Predecir el precio de venta de los inmuebles para vivienda en Bogotá (apartamentos y casas). Para esto se utilizará un algoritmo de ensemble learning y un modelo de redes neuronales. Al final compararemos su performance.

Dado el anterior objetivo, se puede filtrar y eliminar varias de las columnas y filas del dataset para quedarnos solamente con los datos que nos interesan, es decir, los inmuebles que están ubicados en Bogotá, que se ofrecen para la venta y que son únicamente para vivienda.

## Limpieza y filtrado del dataset

In [1]:
# Se importan las librerías a utilizar
import pandas as pd
import numpy as np

In [2]:
# Se lee la base de datos directamente de la página de Properati
# Tenga en cuenta que con el tiempo los datos podrían cambiar ya que el dataset está siendo alimentado constantemente

url = 'https://storage.googleapis.com/properati-data-public/co_properties.csv.gz'

# Se crea un dataframe con los datos
properati = pd.read_csv(url)
properati.head()

,id,ad_type,start_date,end_date,created_on,lat,lon,l1,l2,l3,...,bathrooms,surface_total,surface_covered,price,currency,price_period,title,description,property_type,operation_type
0,KsjahK62rxcYKXXQjOdkqw==,Propiedad,2020-10-07,2021-10-09,2020-10-07,3.9210,-76.506000,Colombia,Valle del Cauca,NaN,...,7.0,NaN,NaN,1.300000e+09,COP,NaN,Casa Campestre en venta en darien 3469064,"HERMOSA CASA CAMPESTRE, &Aacute;REA 6,000 MT, ...",Casa,Venta
1,Y+gsBZYq1zu5NoR3V5oUGA==,Propiedad,2020-10-07,2021-01-06,2020-10-07,3.3577,-76.541811,Colombia,Valle del Cauca,Cali,...,7.0,NaN,NaN,2.800000e+09,COP,NaN,Casa en ciudsd jardin,Casa independiente con posiciona en ciudad jar...,Casa,Venta
2,Jpzqxj8/Vgf3Aa5ASxUBNg==,Propiedad,2020-10-07,2020-10-07,2020-10-07,3.3577,-76.541811,Colombia,Valle del Cauca,Cali,...,7.0,NaN,NaN,2.800000e+09,COP,Mensual,Casa en ciudsd jardin,Casa independiente con posiciona en ciudad jar...,Casa,Venta
3,ieuFnkFx/yHDD66iMV14Gw==,Propiedad,2020-10-07,2021-04-12,2020-10-07,3.3640,-76.538000,Colombia,Valle del Cauca,Cali,...,8.0,NaN,NaN,3.500000e+09,COP,NaN,Casa en venta en pance 1630426,"EXCELENTE CASA - LOTE 6,373 MT, EN OBRA GRIS U...",Casa,Venta
4,g4u5JM+hAHEk8SukRSjMzg==,Propiedad,2020-10-07,9999-12-31,2020-10-07,3.3910,-76.517000,Colombia,Valle del Cauca,Cali,...,9.0,NaN,NaN,4.800000e+08,COP,NaN,CASA EXTERNA BARRIO CIUDAD 2000,"CASA EXTERNA EN EL BARRIO CIUDAD 2000,CONSTRUI...",Casa,Venta


In [3]:
# Vemos la información del dataset: sus columnas, la cantidad de datos faltantes y el tipo de datos
properati.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 25 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   id               1000000 non-null  object 
 1   ad_type          1000000 non-null  object 
 2   start_date       1000000 non-null  object 
 3   end_date         1000000 non-null  object 
 4   created_on       1000000 non-null  object 
 5   lat              740281 non-null   float64
 6   lon              740282 non-null   float64
 7   l1               1000000 non-null  object 
 8   l2               1000000 non-null  object 
 9   l3               937188 non-null   object 
 10  l4               273995 non-null   object 
 11  l5               159127 non-null   object 
 12  l6               54747 non-null    object 
 13  rooms            172087 non-null   float64
 14  bedrooms         376046 non-null   float64
 15  bathrooms        799493 non-null   float64
 16  surface_total    41

## Se caracterizan las columnas
De acuerdo con la información anterior se puede caracterizar cada una de las variables de la siguiente manera: 

* id: código de identificación
* ad_type: identificiación del anuncio (solo hay un tipo)
* start_date: fecha de iniciio del anuncio
* end_date: fecha final del anuncio
* created_on: fecha de creación del anuncio
* lat: latitud del predio
* lon: longitud del predio
* l1: país
* l2: departamento 
* l3: ciudad 
* l4: zona 
* l5: localidad
* l6: barrio
* rooms: no. de espacios 
* bedrooms: no. de habitaciones o dormitorios
* bathrooms: no. de baños
* surface_total: área total en m2
* surface_covered: área privada en m2
* price: precio del inmueble
* currency: moneda
* price_period: perioricidad del precio (aplica para el caso de las propiedades en arriendo)
* title: título del anuncio
* description: breve descripción de las caraccterísticas del inmueble
* property_type: clasifica el tiene de propiedad, tiene varias categorías como: apartamento, casa, oficina, finca, etc.
* operation_type: permite clasificar si el anuncio es para venta o arriendo

In [4]:
# Se listan todos los municipios
# Se ubica la ciudad de Bogotá D.C con 171.160 anuncios de inmuebles tanto en venta como en arriendo y de diferentes tipos de predio

l3_ = pd.DataFrame(properati.groupby(["l3"], sort=True)["l3"].count()
                              .reset_index(name="total"))
l3_.head(30)

,l3,total
0,Abejorral,52
1,Acacías,62
2,Acandí,22
3,Agua de Dios,64
4,Aguazul,18
5,Aipe,2
6,Albania,29
7,Albán,6
8,Alvarado,28
9,Anapoima,623


In [5]:
# Se muestran los diferentes tipos de propiedades registradas en la plataforma

property_type=pd.DataFrame(properati.groupby(["property_type"], sort=True)["property_type"].count()
                              .reset_index(name="total"))
property_type 

,property_type,total
0,Apartamento,565404
1,Casa,220101
2,Depósito,1544
3,Finca,6725
4,Local comercial,26389
5,Lote,46369
6,Oficina,22258
7,Otro,111046
8,Parqueadero,164


In [6]:
# Se listan los diferentes tipos de operación, la mayoría se anuncia en venta
operation_type =pd.DataFrame(properati.groupby(["operation_type"], sort=True)["operation_type"].count()
                              .reset_index(name="total"))
operation_type

,operation_type,total
0,Arriendo,427736
1,Arriendo temporal,380
2,Venta,571884


In [7]:
# Hacemos una primera  eliminación sobre las columnas que ya sabemos que no se necesitan para los objetivos de este análisis
data = properati.drop(columns=['id', 'ad_type', 'start_date', 'end_date', 'created_on','price_period', 'description', 'title'])

#### Filtramos solo las propiedades que nos interesan: 

* ubicación: Bogotá D.C
* tipo de operación: Venta
* tipo de propiedad: Casa y Apartamento 
* currency: Nos interesan sólo las que están en COP, que son la mayoría

In [8]:
data= data[(data.l3 == 'Bogotá D.C') & (data.currency == 'COP') & (data.operation_type == 'Venta') & (data.property_type.isin(['Apartamento','Casa']))]

In [9]:
data.head()

,lat,lon,l1,l2,l3,l4,l5,l6,rooms,bedrooms,bathrooms,surface_total,surface_covered,price,currency,property_type,operation_type
50,NaN,NaN,Colombia,Cundinamarca,Bogotá D.C,Zona Occidental,Engativa,NaN,NaN,NaN,7.0,NaN,NaN,7.600000e+08,COP,Casa,Venta
51,4.648423,-74.084421,Colombia,Cundinamarca,Bogotá D.C,Zona Chapinero,Teusaquillo,NaN,NaN,NaN,7.0,NaN,NaN,2.600000e+09,COP,Casa,Venta
164,4.732122,-74.051263,Colombia,Cundinamarca,Bogotá D.C,Zona Noroccidental,NaN,NaN,3.0,3.0,NaN,NaN,NaN,5.300000e+08,COP,Casa,Venta
165,4.707211,-74.099645,Colombia,Cundinamarca,Bogotá D.C,Zona Occidental,Engativa,NaN,3.0,3.0,NaN,NaN,NaN,5.800000e+08,COP,Casa,Venta
166,4.714907,-74.028475,Colombia,Cundinamarca,Bogotá D.C,Zona Norte,Usaquén,NaN,3.0,3.0,NaN,NaN,NaN,2.200000e+09,COP,Casa,Venta


#### De nuevo sacamos las columnas que no son necesarias ya que son redundantes: país y departamento, pues ya se comprobó que todos los predios están ubicados en Cundinamarca y Colombia

In [10]:
data = data.drop(columns=['l1', 'l2'])

In [11]:
# Damos un vistazo de lo que hemos hecho hasta ahora
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 86262 entries, 50 to 999994
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   lat              81415 non-null  float64
 1   lon              81415 non-null  float64
 2   l3               86262 non-null  object 
 3   l4               85592 non-null  object 
 4   l5               78717 non-null  object 
 5   l6               25716 non-null  object 
 6   rooms            31302 non-null  float64
 7   bedrooms         56799 non-null  float64
 8   bathrooms        67201 non-null  float64
 9   surface_total    4110 non-null   float64
 10  surface_covered  9173 non-null   float64
 11  price            86262 non-null  float64
 12  currency         86262 non-null  object 
 13  property_type    86262 non-null  object 
 14  operation_type   86262 non-null  object 
dtypes: float64(8), object(7)
memory usage: 10.5+ MB


#### Revisamos y contamos los valores faltantes por columna

In [12]:
print("Total Valores Faltantes por Columna")
print(data.isna().sum())

Total Valores Faltantes por Columna
lat                 4847
lon                 4847
l3                     0
l4                   670
l5                  7545
l6                 60546
rooms              54960
bedrooms           29463
bathrooms          19061
surface_total      82152
surface_covered    77089
price                  0
currency               0
property_type          0
operation_type         0
dtype: int64


#### Para efectos del ejercicio que se va a realizar con esta base de datos, nos han solicitado tener como mínimo 1000 datos, entonces para mantener este ejercicio de una manera simple y dado que tenemos una buena cantidad de datos, nos podemos permitir eliminar todas las filas que tienen datos faltantes para enforcarnos en el tema de los modelos aunque lo recomendable siempre es trabajar con la mayor cantidad de datos posibles y tratar de estimar o recuperar estos valores faltantes mediante diferentes técnicas.

In [13]:
data = data.dropna()
data.head()

,lat,lon,l3,l4,l5,l6,rooms,bedrooms,bathrooms,surface_total,surface_covered,price,currency,property_type,operation_type
1311,4.703,-74.057,Bogotá D.C,Zona Noroccidental,Suba,El Batán,5.0,5.0,3.0,175.0,253.0,8.000000e+08,COP,Casa,Venta
1557,4.724,-74.024,Bogotá D.C,Zona Norte,Usaquén,Bosque De Pinos,3.0,3.0,4.0,550.0,369.0,1.800000e+09,COP,Casa,Venta
4291,4.728,-74.046,Bogotá D.C,Zona Norte,Usaquén,Cedritos,2.0,2.0,1.0,50.0,50.0,2.400000e+08,COP,Apartamento,Venta
4307,4.634,-74.064,Bogotá D.C,Zona Chapinero,Chapinero,Chapinero Central,1.0,1.0,1.0,31.0,31.0,1.800000e+08,COP,Apartamento,Venta
5764,4.727,-74.036,Bogotá D.C,Zona Norte,Usaquén,Cedritos,1.0,1.0,2.0,74.0,74.0,5.500000e+08,COP,Apartamento,Venta


In [14]:
# Se han eliminado todos los datos faltantes
print(data.isna().sum())

lat                0
lon                0
l3                 0
l4                 0
l5                 0
l6                 0
rooms              0
bedrooms           0
bathrooms          0
surface_total      0
surface_covered    0
price              0
currency           0
property_type      0
operation_type     0
dtype: int64


#### Se observa que la columna rooms y bedrooms parecen tener los mismos valores. Para comprobar lo anterior con la siguiente línea de código se hará una comparación de cada uno de los valores de todas las columnas en el dataset y al final nos arrojará las columnas que tienen valores duplicados.

In [15]:
# Comprobar duplicados

duplicates = []
for col in range(data.shape[1]):
    contents = data.iloc[:, col]
    
    for comp in range(col + 1, data.shape[1]):
        if contents.equals(data.iloc[:, comp]):
            duplicates.append(comp)
duplicates = np.unique(duplicates).tolist()

In [16]:
# Listamos las columnas duplicadas
duplicates

[7]

#### Definitivamente se comprueba que la columna bedrooms está duplicada, procedemos a eliminarla

In [17]:
data=data.drop(data.columns[duplicates], axis=1)

In [18]:
# De nuevo, revisamos la información sobre el nuevo dataset, nos hemos quedado con 1802 datos y 14 columnas
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1802 entries, 1311 to 999876
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   lat              1802 non-null   float64
 1   lon              1802 non-null   float64
 2   l3               1802 non-null   object 
 3   l4               1802 non-null   object 
 4   l5               1802 non-null   object 
 5   l6               1802 non-null   object 
 6   rooms            1802 non-null   float64
 7   bathrooms        1802 non-null   float64
 8   surface_total    1802 non-null   float64
 9   surface_covered  1802 non-null   float64
 10  price            1802 non-null   float64
 11  currency         1802 non-null   object 
 12  property_type    1802 non-null   object 
 13  operation_type   1802 non-null   object 
dtypes: float64(7), object(7)
memory usage: 211.2+ KB


#### A efectos de tener mayor claridad sobre cada variable y por facilidad, procedemos a renombrarlas

In [19]:
# Renombramos las columnas
data.columns = ['lat','lon','ciudad','zona', 'localidad', 'barrio', 'habitaciones', 'baños', 'area', 'area_privada', 'precio', 'moneda',
                'tipo_propiedad', 'tipo_operacion']

In [20]:
data.head()

,lat,lon,ciudad,zona,localidad,barrio,habitaciones,baños,area,area_privada,precio,moneda,tipo_propiedad,tipo_operacion
1311,4.703,-74.057,Bogotá D.C,Zona Noroccidental,Suba,El Batán,5.0,3.0,175.0,253.0,8.000000e+08,COP,Casa,Venta
1557,4.724,-74.024,Bogotá D.C,Zona Norte,Usaquén,Bosque De Pinos,3.0,4.0,550.0,369.0,1.800000e+09,COP,Casa,Venta
4291,4.728,-74.046,Bogotá D.C,Zona Norte,Usaquén,Cedritos,2.0,1.0,50.0,50.0,2.400000e+08,COP,Apartamento,Venta
4307,4.634,-74.064,Bogotá D.C,Zona Chapinero,Chapinero,Chapinero Central,1.0,1.0,31.0,31.0,1.800000e+08,COP,Apartamento,Venta
5764,4.727,-74.036,Bogotá D.C,Zona Norte,Usaquén,Cedritos,1.0,2.0,74.0,74.0,5.500000e+08,COP,Apartamento,Venta


In [21]:
# Cambiamos el índice para empezar desde cero
data.reset_index(inplace=True, drop=True)
data.head()

,lat,lon,ciudad,zona,localidad,barrio,habitaciones,baños,area,area_privada,precio,moneda,tipo_propiedad,tipo_operacion
0,4.703,-74.057,Bogotá D.C,Zona Noroccidental,Suba,El Batán,5.0,3.0,175.0,253.0,8.000000e+08,COP,Casa,Venta
1,4.724,-74.024,Bogotá D.C,Zona Norte,Usaquén,Bosque De Pinos,3.0,4.0,550.0,369.0,1.800000e+09,COP,Casa,Venta
2,4.728,-74.046,Bogotá D.C,Zona Norte,Usaquén,Cedritos,2.0,1.0,50.0,50.0,2.400000e+08,COP,Apartamento,Venta
3,4.634,-74.064,Bogotá D.C,Zona Chapinero,Chapinero,Chapinero Central,1.0,1.0,31.0,31.0,1.800000e+08,COP,Apartamento,Venta
4,4.727,-74.036,Bogotá D.C,Zona Norte,Usaquén,Cedritos,1.0,2.0,74.0,74.0,5.500000e+08,COP,Apartamento,Venta


#### Guardamos nuestro nuevo dataset en un nuevo fichero .csv

Con este dataset se desarrollará el resto de la actividad en un nuevo notebook. Tanto el notebook de la actividad 'regresion_mercadoInmobiliario.ipynb' como el nuevo dataset que se acaba de generar 'properties_bog.csv' se encuentran alojados en el repositorio de <a href='https://github.com/juli-amezquita/Precios-vivienda-Bogota'>Github.</a> 

In [22]:
data.to_csv('properties_bog.csv', sep=';')